In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
import math
import pydotplus
from sklearn import tree
import collections


from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

/Users/yinchiahuang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Prepare data set

In [2]:
# Take your stock, take 2017(52 weeks) + 2018(20 weeks)
HMC = pd.read_csv('/Users/yinchiahuang/Library/Mobile Documents/com~apple~CloudDocs/CS677/Assignments/yinchia@bu.edu_hw1/HMC_Honda Motor Co., Ltd.csv')
HMC_1718 = HMC.copy()
HMC_1718['Date'] = pd.to_datetime(HMC_1718['Date'])
start_date = '12-28-2016'
end_date = '05-18-2018'
mask = (HMC_1718['Date'] > start_date) & (HMC_1718['Date'] <= end_date)
HMC_1718 = HMC_1718.loc[mask]
HMC_1718['Return'] = HMC_1718['Adj Close'].pct_change()
HMC_1718['Day of week'] = HMC_1718['Date'].dt.weekday_name

In [3]:
# find out the missing data --> make sure every week has monday to friday
date_index = HMC_1718["Date"].tolist()
date_index2 = pd.date_range('12-28-2016', periods= 507, freq='D')

HMC_1718_returns = pd.DataFrame({"Daily return": HMC_1718["Return"].tolist(), "Day of week":HMC_1718["Day of week"].tolist()}, index = date_index)
HMC_1718_returns = HMC_1718_returns.reindex(date_index2)

In [4]:
# get vectors of weekly returns
weekly_returns = [] 

HMC_1718_returns = HMC_1718_returns.reset_index()

for row in range(5,len(HMC_1718_returns)):
    daily_return = HMC_1718_returns.loc[row]['Daily return'] 
    
    # replace missing value with previous data
    r = row -1
    while np.isnan(daily_return)== True:
        daily_return = HMC_1718_returns.loc[r]['Daily return']
        r -=1
        
    if row%7 == 5: # Monday --> create a new vector & add daily return in it 
        weekly_return = []
        weekly_return.append(daily_return)
    elif row%7 == 6 or row%7 == 0 or row%7 == 1: # Tues, Wed, Thurs --> add daily return in the vector

        weekly_return.append(daily_return)
    elif row%7 == 2 :# Friday --> add daily return in the vector & add the vector to a list of all vectors  
        weekly_return.append(daily_return)
        weekly_return = [round(elem, 4) for elem in weekly_return]
        weekly_returns.append(weekly_return)


In [5]:
# assign labels for each week --> I define a week is good(1) if this weeks' Fri return > Mon return, otherwise, it is a bad(0) week
week_labels = []

for i in range(len(weekly_returns)):
    monday_return = weekly_returns[i][0]
    friday_return = weekly_returns[i][4]

    if friday_return>= monday_return:
        week_labels.append(1)
    else:
        week_labels.append(0)

In [6]:
# get daily returns as attributes
monday_returns = []
tuesday_returns = []
wednesday_returns = []
thursday_returns = []
friday_returns = [] 

for week in range(len(weekly_returns)):
    monday_returns.append(weekly_returns[week][0])
    tuesday_returns.append(weekly_returns[week][1])
    wednesday_returns.append(weekly_returns[week][2])
    thursday_returns.append(weekly_returns[week][3])
    friday_returns.append(weekly_returns[week][4])

In [7]:
#summarize all information
data = {'Mon returns':monday_returns,'Tue returns':tuesday_returns,'Wed returns':wednesday_returns,'Thu returns':thursday_returns,'Fri returns':friday_returns, 'daily returns':weekly_returns,'week label': week_labels}
df = pd.DataFrame(data=data)
df

,Mon returns,Tue returns,Wed returns,Thu returns,Fri returns,daily returns,week label
0,-0.0031,0.0144,0.0355,0.0000,-0.0062,"[-0.0031, 0.0144, 0.0355, 0.0, -0.0062]",0
1,-0.0013,-0.0108,0.0027,-0.0007,-0.0027,"[-0.0013, -0.0108, 0.0027, -0.0007, -0.0027]",0
2,-0.0027,-0.0140,0.0111,0.0120,0.0030,"[-0.0027, -0.014, 0.0111, 0.012, 0.003]",1
3,-0.0026,-0.0016,0.0158,-0.0019,-0.0212,"[-0.0026, -0.0016, 0.0158, -0.0019, -0.0212]",0
4,-0.0043,-0.0077,0.0081,0.0087,0.0278,"[-0.0043, -0.0077, 0.0081, 0.0087, 0.0278]",1
5,-0.0061,0.0165,0.0112,-0.0126,0.0077,"[-0.0061, 0.0165, 0.0112, -0.0126, 0.0077]",1
6,0.0177,-0.0037,-0.0069,-0.0091,0.0035,"[0.0177, -0.0037, -0.0069, -0.0091, 0.0035]",0
7,0.0035,0.0054,0.0025,-0.0075,-0.0054,"[0.0035, 0.0054, 0.0025, -0.0075, -0.0054]",0
8,-0.0067,-0.0086,0.0103,-0.0096,0.0052,"[-0.0067, -0.0086, 0.0103, -0.0096, 0.0052]",1
9,-0.0061,0.0036,-0.0042,0.0000,0.0036,"[-0.0061, 0.0036, -0.0042, 0.0, 0.0036]",1


In [8]:
# Training data set
df_2017 = df[:52]

data_features = ['Mon returns', 'Tue returns', 'Wed returns','Thu returns','Fri returns']

# Test data set
df_2018 = df[52:72]
df_2018 = df_2018.reset_index()
df_2018 = df_2018.drop('index', 1)


In [9]:
#df_2017

In [10]:
#df_2018

#### 1.	Logistic regression

In [11]:
X_train = df_2017.iloc[:,0:5]
y_train = df_2017.iloc[:, 6]

X_test = df_2018.iloc[:,0:5]
y_test = df_2018.iloc[:,6]

# deploy logistic regression
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

y_pred = LogReg.predict(X_test)
LogReg_predicteds = y_pred.tolist()
LogReg_predicteds

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### 2. KNN (k =5) : Predicted label by k nearest neighbor classification

In [12]:
from scipy.spatial import distance

def getKNNPredictedLabel(df,starting_row,k):
    distance_euclidean = {}
    a = (df.loc[starting_row]['Mon returns'],
         df.loc[starting_row]['Tue returns'],
         df.loc[starting_row]['Wed returns'],
         df.loc[starting_row]['Thu returns'],
         df.loc[starting_row]['Fri returns'])
    
    for i in range(1,6):
        b = (df.loc[starting_row - i]['Mon returns'],
             df.loc[starting_row - i]['Tue returns'],
             df.loc[starting_row - i]['Wed returns'],
             df.loc[starting_row - i]['Thu returns'],
             df.loc[starting_row - i]['Fri returns'])
        dst_euclidean = distance.euclidean(a,b)
        distance_euclidean[i]= dst_euclidean

    sorted_distance_euclidean = sorted(distance_euclidean, key=distance_euclidean.get)
    neareast_i = sorted_distance_euclidean[:k]
    neareast_rows = [starting_row - i for i in neareast_i]
    predicted_labels = [df.loc[row]['week label'] for row in neareast_rows]
    predicted_label = 1 if np.bincount(predicted_labels).argmax()==1 else 0
    return predicted_label

In [13]:
knn_predicted_labels = []
# from week6 to week 71
for row in range(6, 72):
    knn_predicted_label = getKNNPredictedLabel(df,row,5)
    knn_predicted_labels.append(knn_predicted_label)

# 2018 predicted labels is from week51 to week 71
knn_predicteds = knn_predicted_labels[-20:]
knn_predicteds

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### 3.	naïve Bayesian (assume independent and Gaussian distributions for returns

In [14]:
gnb = GaussianNB()

gnb.fit(df_2017[data_features].values, df_2017["week label"])
NB_predicted = gnb.predict(df_2018[data_features])
NB_predicteds = NB_predicted.tolist()
NB_predicteds

[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

#### 4. K-means 
#### Take k = 3, find the nearest centroid for your new vectors, assign the label to the new vector based on the majority of labels in that cluster

In [15]:
# classify each day to clusters

from sklearn.cluster import KMeans
X = np.asarray(df_2017["daily returns"].tolist())

km = KMeans(n_clusters=3, init='random', 
            n_init=10, max_iter=300, tol=1e-04,
            random_state=0)

km_predicted = km.fit_predict(X)

clusters = [label+1 for label in km_predicted]
clusters

data = {'daily returns': df_2017["daily returns"].tolist(),'week label': df_2017["week label"].tolist(),'clusters':clusters}
df_cluster_2017 = pd.DataFrame(data=data)

In [16]:
# get the majority of labels in each cluster
good_cluster1 = 0 
good_cluster2 = 0
good_cluster3 = 0

bad_cluster1 = 0 
bad_cluster2 = 0
bad_cluster3 = 0

for row in range(len(df_cluster_2017)):
    if df_cluster_2017.loc[row]['clusters'] == 1:
        if df_cluster_2017.loc[row]['week label'] == 1:
            good_cluster1 +=1
        else:
            bad_cluster1 +=1
    elif df_cluster_2017.loc[row]['clusters'] == 2:
        if df_cluster_2017.loc[row]['week label'] == 1:
            good_cluster2 +=1
        else:
            bad_cluster2 +=1        
    else:
        if df_cluster_2017.loc[row]['week label'] == 1:
            good_cluster3 +=1
        else:
            bad_cluster3 +=1

total_cluster1 = good_cluster1+bad_cluster1
total_cluster2 = good_cluster2+bad_cluster2
total_cluster3 = good_cluster3+bad_cluster3

print("cluster1: # of good weeks= {0}({3}%), # of bad weeks= {1}({4}%), # of total weeks= {2}(100%)".format(good_cluster1,bad_cluster1,total_cluster1,round(good_cluster1/total_cluster1*100,2),round(bad_cluster1/total_cluster1*100,2)))
print("cluster2: # of good weeks= {0}({3}%), # of bad weeks= {1}({4}%), # of total weeks= {2}(100%)".format(good_cluster2,bad_cluster2,total_cluster2,round(good_cluster2/total_cluster2*100,2),round(bad_cluster2/total_cluster2*100,2)))
print("cluster3: # of good weeks= {0}({3}%), # of bad weeks= {1}({4}%), # of total weeks= {2}(100%)".format(good_cluster3,bad_cluster3,total_cluster3,round(good_cluster3/total_cluster3*100,2),round(bad_cluster3/total_cluster3*100,2)))


cluster1: # of good weeks= 12(37.5%), # of bad weeks= 20(62.5%), # of total weeks= 32(100%)
cluster2: # of good weeks= 0(0.0%), # of bad weeks= 3(100.0%), # of total weeks= 3(100%)
cluster3: # of good weeks= 11(64.71%), # of bad weeks= 6(35.29%), # of total weeks= 17(100%)


- cluster1: bad (0)
- cluster2: bad (0)
- cluster3: good (1)

In [17]:
Mon_centers = km.cluster_centers_[:, 0]
Tue_centers = km.cluster_centers_[:, 1]
Wed_centers = km.cluster_centers_[:, 2]
Thu_centers = km.cluster_centers_[:, 3]
Fri_centers = km.cluster_centers_[:, 4]

#cluster centers
center_cluster1 = [Mon_centers[0],Tue_centers[0],Wed_centers[0],Thu_centers[0],Fri_centers[0]]
center_cluster2 = [Mon_centers[1],Tue_centers[1],Wed_centers[1],Thu_centers[1],Fri_centers[1]]
center_cluster3 = [Mon_centers[2],Tue_centers[2],Wed_centers[2],Thu_centers[2],Fri_centers[2]]

def getKMPredictedLabel(df,starting_row):
    a = (df.loc[starting_row]['daily returns'])
    distance_cluster1 = distance.euclidean(a,center_cluster1)
    distance_cluster2 = distance.euclidean(a,center_cluster2)
    distance_cluster3 = distance.euclidean(a,center_cluster3)
    
    nearest_centroid = min(distance_cluster1,distance_cluster2,distance_cluster3) 

    if nearest_centroid == distance_cluster1 or nearest_centroid == distance_cluster2:
        KM_predicted = 0
    else:
        KM_predicted = 1
    
    return KM_predicted


In [18]:
KM_predicteds = []
# test 2018 data
for row in range(len(df_2018)):
    KM_predicted = getKMPredictedLabel(df_2018,row)
    KM_predicteds.append(KM_predicted)

KM_predicteds

[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1]

#### 5. Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression

xs = df_2017['daily returns'].tolist()
ys = df_2017['week label'].tolist()
ys = np.array(ys)[:,np.newaxis]

LinearRmodel = LinearRegression(fit_intercept=True)
LinearRmodel.fit(xs, ys)


# test 2018 data
LinearR_predicteds = []
for row in range(len(df_2018)):
    predicted_y = LinearRmodel.predict([df.loc[row]['daily returns']])
    distance_1 = abs(predicted_y-1)
    distance_0 = abs(predicted_y-0)
    LinearR_predicted = 1 if distance_1< distance_0 else 0
    LinearR_predicteds.append(LinearR_predicted)

LinearR_predicteds

[0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0]

#### 6. Decision tree (use entropy)

In [20]:
X_train = df_2017[['Mon returns', 'Tue returns', 'Wed returns','Thu returns','Fri returns']].values
Y_train = df_2017[['week label']].values
clf = tree.DecisionTreeClassifier(criterion = 'entropy')
clf = clf.fit(X_train,Y_train)

In [21]:
# visualize the decision tree
dot_data = tree.export_graphviz(clf,
                                feature_names=data_features,
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)


colors = ('palegreen', 'salmon')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])
        
input_dir = '/Users/yinchiahuang/Library/Mobile Documents/com~apple~CloudDocs/CS677/Assignments/yinchia@bu.edu_hw12'
file_name = os.path.join(input_dir, 'tree_continuous.pdf')
graph.write_pdf(file_name)

True

In [22]:
X_test = df_2018[['Mon returns', 'Tue returns', 'Wed returns','Thu returns','Fri returns']].values
Y_test = df_2018[['week label']].values
DSTree_predicteds = clf.predict(X_test)  
DSTree_predicteds= DSTree_predicteds.tolist()
DSTree_predicteds

[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0]

#### 7.Support vector machines:
- (a) Linear
- (b) Radial basis function (Gaussian)
- (c) Optional: polynomial degree 3

In [23]:
from sklearn import svm, datasets

def SVM(X_train,Y_train,X_test,Y_test,c,gamma,kernel_name):
    if kernel_name == 'rbf':
        model = svm.SVC(kernel='rbf', gamma=gamma, C=C).fit(X_train,Y_train)
    elif kernel_name == 'poly':
        model = svm.SVC(kernel='poly', C=C, degree=degree).fit(X_train,Y_train)
    elif kernel_name =='linear':
        model = svm.LinearSVC().fit(X_train,Y_train)
    else:
        model = svm.SVC(kernel=kernel_name).fit(X_train,Y_train)
    
    predicted = model.predict(X_test)
    return predicted

In [24]:
C = 1.0
gamma = 1
#C = 2
degree = 2
#kernel_name = 'linear'
#C=0.5

data_features = ['Mon returns', 'Tue returns', 'Wed returns','Thu returns','Fri returns']
X_train = df_2017[data_features].values
Y_train = df_2017['week label'].values
X_test = df_2018[data_features].values
Y_test = df_2018['week label'].values

SVM_linear_predicteds = SVM(X_train,Y_train,X_test,Y_test,C,gamma,'linear')
SVM_rbf_predicteds = SVM(X_train,Y_train,X_test,Y_test,C,gamma,'rbf')


#### Represent each week as a single number: closing price on Friday and consider the following algorithms:
- 1. Naïve (next week is the same as this week)
- 2. Moving average (window = 10, use closing price)
- 3. Bollinger band (window = 10, k= 2)

#### Prepare data

In [25]:
# find out the missing data --> make sure every week has monday to friday
date_index = HMC_1718["Date"].tolist()
date_index2 = pd.date_range('12-28-2016', periods= 507, freq='D')

HMC_1718_prices = pd.DataFrame({"Close":HMC_1718["Close"].tolist(),"Day of week":HMC_1718["Day of week"].tolist()}, index = date_index)
HMC_1718_prices = HMC_1718_prices.reindex(date_index2)
#HMC_1718_prices

In [26]:
# get vectors of friday close prices
friday_closes = [] 

HMC_1718_prices = HMC_1718_prices.reset_index()

for row in range(5,len(HMC_1718_prices)):
    friday_close = HMC_1718_prices.loc[row]['Close'] 
    
    # replace missing value with previous data
    r = row -1
    while np.isnan(friday_close)== True:
        friday_close = HMC_1718_prices.loc[r]['Close']
        r -=1
    if row%7 ==2: # Friday
        friday_closes.append(round(friday_close,2))

In [27]:
#summarize all information
data = {'Friday close':friday_closes,'week label': week_labels}
df2 = pd.DataFrame(data=data)
df2

,Friday close,week label
0,30.47,0
1,30.08,0
2,30.44,1
3,30.08,0
4,31.06,1
5,31.57,1
6,31.61,0
7,31.45,0
8,31.15,1
9,31.05,1


#### 1. Naïve (next week is the same as this week)

In [28]:
df2['Naïve_predicted'] = np.where(df2['Friday close'] >= df2['Friday close'].shift(+1),1,0)
Naïve_predicteds = df2['Naïve_predicted'].tolist()

# 2018 predicted labels is from week51 to week 71
Naïve_predicteds = Naïve_predicteds[-20:]
Naïve_predicteds

[1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]

#### 2. Moving average (window = 10, use closing price)
- Buy (Good label) : st_ma > lt_ma
- Sell (Bad label) : st_ma < lt_ma 
- Ex .Just use 10 weeks to make a decision for new week. In the case of 2017, take last 10 weeks of 2017 and prodict label for week 1 for 2018. Then, take 9 weeks of 2017 and one week of 2018 to predict week 2 of 2018 and so on

In [29]:
window =10
df2['ma'] = df2['Friday close'].rolling(window).mean()
df2['MA_predicted'] =  np.where(df2['Friday close'] >= df2['ma'].shift(+1),1,0)
MA_predicteds = df2['MA_predicted'].tolist()[-20:]
MA_predicteds

[1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

#### 3. Bollinger band (window = 10, k= 2)
- Buy (Good label) : price > upper band
- Sell (Bad label) : price < lower band

In [30]:
window =10
k = 2 
df2['std'] = df2['Friday close'].rolling(window).std() 
df2['lower_band'] = df2['ma']- k* df2['std']
df2['upper_band'] = df2['ma'] + k* df2['std']
df2['BB_predicted'] =  np.where(df2['Friday close'] >= df2['upper_band'],1,0)
BB_predicteds = df2['BB_predicted'].tolist()[-20:]
BB_predicteds

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### Summarize all methods in a table:

In [31]:
def getPerformance(classifier, predicted_list,actual_list):
    number_of_good = 0
    number_of_bad = 0
    correct_good = 0
    correct_bad = 0 
    for i in range(len(actual_list)):
        # good label
        if predicted_list[i] == 1:
            number_of_good +=1
            # classified correct
            if predicted_list[i] == actual_list[i]:
                correct_good +=1            
        # bad label    
        else:
            number_of_bad +=1
            # classified correct
            if predicted_list[i] == actual_list[i]:
                correct_bad +=1

    pct_correct_good = 100* correct_good/number_of_good if number_of_good !=0 else 0
    pct_correct_bad = 100* correct_bad/number_of_bad if number_of_bad !=0 else 0 

    print("{:20s}{:15.2f}%{:10d}{:15.2f}%{:11d}"
          .format(classifier,pct_correct_good,number_of_good,pct_correct_bad,number_of_bad ))
            

In [32]:
actual_labels = week_labels[-20:]
print("Actual labels: # of '+' is ", sum(actual_labels), "; # of 'bad' is ", len(actual_labels)-sum(actual_labels))

print("\n{:23s}{:19s}{:11s}{:16s}{:12s}"
    .format("","correct for +","# of +","correct for -","# of -"))
getPerformance('Logistic Regression',LogReg_predicteds,actual_labels)
getPerformance('KNN',knn_predicteds,actual_labels)
getPerformance('Naïve Bayesian',NB_predicteds,actual_labels)
getPerformance('K means',KM_predicteds,actual_labels)
getPerformance('Linear Regression',LinearR_predicteds,actual_labels)
getPerformance('Decision Tree',DSTree_predicteds,actual_labels)
getPerformance('SVM_Linear',SVM_linear_predicteds,actual_labels)
getPerformance('SVM_RBF',SVM_rbf_predicteds,actual_labels)
getPerformance('Naïve',Naïve_predicteds,actual_labels)
getPerformance('Moving Average',MA_predicteds,actual_labels)
getPerformance('Bollinger Band',BB_predicteds,actual_labels)

Actual labels: # of '+' is  9 ; # of 'bad' is  11

                       correct for +      # of +     correct for -   # of -      
Logistic Regression            0.00%         0          55.00%         20
KNN                           50.00%         6          57.14%         14
Naïve Bayesian               100.00%         6          78.57%         14
K means                       57.14%         7          61.54%         13
Linear Regression             55.56%         9          63.64%         11
Decision Tree                 81.82%        11         100.00%          9
SVM_Linear                     0.00%         0          55.00%         20
SVM_RBF                        0.00%         0          55.00%         20
Naïve                         33.33%         9          45.45%         11
Moving Average                62.50%         8          66.67%         12
Bollinger Band               100.00%         1          57.89%         19


#### Conclusion
- The best alogrithm for this data set is Decision Tree. Because it has enough data classified for each label. And the correctness rate are as high as 81.82% and 100%.
- The worst algorithms are Logistic Regression, SVM, and Naïve. 